In [5]:
import urllib3
http=urllib3.PoolManager()
r=http.request('GET', 'https://raw.githubusercontent.com/dwyl/english-words/master/words.txt', \
              preload_content=False)
print(r.status) #200 means request is succeeded.
preload_content=False
with open('englishwords.txt', 'wb') as engw:
    data=r.read()
    engw.write(data)
r.release_conn()

200


In [1]:
#Data cleaning and filtering words of length 3, starts with a small letter
word_list=open('englishwords.txt').readlines()
word_list=map(str.strip, word_list)
req_words=[word for word in word_list if len(word)==3]
req_words=[word for word in req_words if word[0].islower()]
req_words=[word for word in req_words if word.isalpha()]
req_words=list(map(str.lower, req_words))
req_words

['aah',
 'aal',
 'abb',
 'abd',
 'aby',
 'abl',
 'abn',
 'abr',
 'abt',
 'abv',
 'acy',
 'ade',
 'ady',
 'adj',
 'ado',
 'adv',
 'adz',
 'aeq',
 'aer',
 'afd',
 'aff',
 'aga',
 'age',
 'agy',
 'ago',
 'ahi',
 'aho',
 'ahs',
 'ahu',
 'aye',
 'aik',
 'ail',
 'ays',
 'ait',
 'ayu',
 'ake',
 'ako',
 'aku',
 'ale',
 'aly',
 'alk',
 'all',
 'aln',
 'alt',
 'alw',
 'ana',
 'and',
 'ane',
 'ans',
 'ant',
 'aob',
 'aor',
 'aph',
 'apx',
 'arb',
 'ard',
 'ary',
 'arn',
 'arr',
 'art',
 'arx',
 'asb',
 'ase',
 'ast',
 'ate',
 'aud',
 'auf',
 'auh',
 'aul',
 'avg',
 'avn',
 'awa',
 'awd',
 'awe',
 'awm',
 'awn',
 'azo',
 'bad',
 'bah',
 'baw',
 'bbs',
 'bcf',
 'bdl',
 'bec',
 'beg',
 'bey',
 'bet',
 'bhd',
 'bye',
 'big',
 'bin',
 'bio',
 'byp',
 'bys',
 'biz',
 'bkg',
 'bks',
 'bkt',
 'bld',
 'ble',
 'blk',
 'blo',
 'boa',
 'boe',
 'bog',
 'boh',
 'boo',
 'bpt',
 'brr',
 'bsh',
 'buy',
 'bun',
 'buz',
 'bvt',
 'bxs',
 'cag',
 'cay',
 'caw',
 'ccm',
 'cdg',
 'cee',
 'cep',
 'cfh',
 'cfm',
 'cfs',


In [2]:
#Sorting words
import numpy as np
word_list=np.array(req_words)
word_list.dtype #Checking dtype of word_list
word_list.sort()
word_list

array(['aah', 'aal', 'abb', ..., 'zod', 'zoo', 'zzt'], dtype='<U3')

In [5]:
#Converting words into bytes
words=[]
for word in word_list:
    words.extend(ord(ele) for ele in word)
words=np.array(words)
words=words.reshape((1018,3))
words

array([[ 97,  97, 104],
       [ 97,  97, 108],
       [ 97,  98,  98],
       ...,
       [122, 111, 100],
       [122, 111, 111],
       [122, 122, 116]])

In [6]:
#Calculating hamming distance
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
#pdist calculates pairwise hamming distance in n_D space
hamming_distance=pdist(words, metric='hamming')
print(hamming_distance.shape)
((1018*1018)-1018)/2 #checking shape of hamming_distance 

(517653,)


517653.0

In [7]:
#Squreform: 
squareform(hamming_distance)[0:4, 0:4]
mat=squareform(hamming_distance<1.5/3)
mat.shape

(1018, 1018)

In [8]:
graph=csr_matrix(mat)

In [9]:
#Here, our first word is "and" & second word is "dog"
i1=word_list.searchsorted('and')
i2=word_list.searchsorted('dog')
word_list[i1]

'and'

In [10]:
from scipy.sparse.csgraph import dijkstra
distances, predecessors=dijkstra(graph, indices=i1, return_predecessors=True)
print(distances[i2])
print(predecessors)

5.0
[  64   65    3 ...  936  727 1013]


In [12]:
#we get the words from "and" to "dog" that differ by one letter
path=[]
i=i2
while(i!=i1):
    path.append(word_list[i])
    i=predecessors[i]
path.append(word_list[i1])
print(path[::-1])

['and', 'aud', 'mud', 'mug', 'dug', 'dog']


# EXPLORING FURTHER 

In [13]:
from scipy.sparse.csgraph import connected_components
N_components, component_list=connected_components(graph)
print(N_components)
print(component_list.shape)
component_list

9
(1018,)


array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [14]:
[np.sum(component_list==i) for i in range(N_components)]

[1010, 1, 1, 1, 1, 1, 1, 1, 1]

In [16]:
[list(word_list[np.where(component_list==i)]) for i in range(1, N_components)]

[['bcf'], ['bdl'], ['epi'], ['mmf'], ['sml'], ['wjc'], ['xcl'], ['xyz']]

In [17]:
distances, predecessors=dijkstra(graph, return_predecessors=True)
max_distance=np.max(distances[~np.isinf(distances)])
print(max_distance)

11.0


In [23]:
i1, i2=np.where(distances==max_distance)
list(zip(word_list[i1], word_list[i2]))

[('dlr', 'hmm'),
 ('dlr', 'kmc'),
 ('hmm', 'dlr'),
 ('hmm', 'lbf'),
 ('hmm', 'rfz'),
 ('hmm', 'tfr'),
 ('kmc', 'dlr'),
 ('kmc', 'tfr'),
 ('lbf', 'hmm'),
 ('rfz', 'hmm'),
 ('tfr', 'hmm'),
 ('tfr', 'kmc')]

In [27]:
for j, i in zip(i1, i2):
    path=[]
    while(i!=j):
        path.append(word_list[i])
        i=predecessors[j,i]
    path.append(word_list[j])
    print(path[::-1])

['dlr', 'tlr', 'tlo', 'blo', 'boo', 'boh', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['dlr', 'tlr', 'tlo', 'blo', 'bld', 'bad', 'bah', 'pah', 'pph', 'kph', 'kpc', 'kmc']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ugt', 'tgt', 'tot', 'too', 'tlo', 'tlr', 'dlr']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ast', 'abt', 'abv', 'dbv', 'dbw', 'lbw', 'lbf']
['hmm', 'umm', 'ump', 'unp', 'uns', 'ons', 'ous', 'ouf', 'suf', 'suz', 'sfz', 'rfz']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ugt', 'tgt', 'tot', 'too', 'tlo', 'tlr', 'tfr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'dlr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'tfr']
['lbf', 'lbw', 'dbw', 'daw', 'baw', 'bah', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['rfz', 'rfb', 'rhb', 'rho', 'sho', 'shi', 'ihi', 'imi', 'imu', 'umu', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'blo', 'boo', 'boh', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'blo', 'bld', 'bad', 'bah', 'pah', 'pph', '